# SNLP Assignment 8

Name 1: Entang Wang<br/>
Student id 1: 7069521<br/>
Email 1: enwa00001@stud.uni-saarland.de<br/>


Name 2: <br/>
Student id 2: <br/>
Email 2: <br/> 

Name 3: <br/>
Student id 3: <br/>
Email 3: <br/> 

**Instructions:** Read each question carefully. <br/>
Make sure you appropriately comment your code wherever required. Your final submission should contain the completed Notebook. There is no need to submit the data files. <br/>
Upload the zipped folder on CMS. Please follow the naming convention of **Name1_studentID1_Name2_studentID2_Name3_studentID3.zip**. Make sure to click on "Turn-in" (or the equivalent on CMS) after you upload your submission, otherwise the assignment will not be considered as submitted. Only one member of the group should make the submisssion.

---

In this Assignment you will be asked to draw graphs, You can do this by hand or use an online tool like https://app.diagrams.net/

Just make sure to submit the graphs as images and specify the question they are answering.

# Exercise 1 $\chi^2$ feature selection 1 Points
Consider a setting where the task is to classify a collection of scientific articles based on their topics. The table below shows some word occurrence statistics from this document collection for four topic categories: mathematics, chemistry, astronomy, and physics.

|                 | N(composition) | N(gravity) | N(differential) | N(theory) |
|-----------------|----------------|------------|-----------------|-----------|
| c = mathematics | 30             | 3          | 50              | 7         |
| c = chemistry   | 43             | 0          | 2               | 5         |
| c = astronomy   | 47             | 53         | 19              | 11        |
| c = physics     | 30             | 34         | 29              | 7         |

1. Explain how we can use $\chi^2$ statistics to asses wether each word feature would be useful for predicting the class. (0.5 Points)
2. Implement the function chi2_statistic and compute the $\chi^2$ statistics for the given features. (0.5 Points)

## Answer 1.1
The $\chi^2$  test is a statistical method used to test whether two categorical variables are independent. In test classification, the category variable is the class label, and the feature variable is the word occurrence. The null hypothesis is that the word occurrence is independent of the class label, meaning that in different classes, the word occurrence should be similar. The $\chi^2$ statistic measures the difference between the observed frequency of word occurrences in each class and the expected frequency if the two variables were independent. A high $\chi^2$ value indicates that the word occurrence is not independent of the class label, suggesting that the word feature is useful for predicting the class. In general, word features with high $\chi^2$ values are more likely to be useful for classification tasks.

In [1]:
import numpy as np
contingency_table = np.array([[30 ,3 ,50,7],#math
                     [43,0,2,5],#chem
                     [47,53,19,11],#astr
                     [30,34,29,7]])#phys

In [ ]:
def chi2_statistic(contingency_table: list[list[int]]) -> float:
    """
    Compute the chi-squared statistic from a contingency table.

    Parameters:
        contingency_table (list of list of int):
            A 2D table where entry [i][j] is the count of samples with
            feature value i and label j.

    Returns:
        float: The chi-squared statistic.
    """

    observed_table = np.array(contingency_table)

    row_totals = observed_table.sum(axis=1, keepdims=True)
    col_totals = observed_table.sum(axis=0, keepdims=True)
    total = observed_table.sum()

    expected_table = np.outer(row_totals, col_totals) / total
    squared_diff = (observed_table - expected_table) ** 2
    chi2_terms = np.divide(squared_diff, expected_table, out=np.zeros_like(squared_diff), where=expected_table != 0)
    chi2_stat = chi2_terms.sum()
    return chi2_stat


words = ["composition", "gravity", "differential", "theory"]
chi2_scores = {}
for i, word in enumerate(words):
    present_counts = contingency_table[:, i]
    absent_counts = contingency_table.sum(axis=1) - present_counts
    feature_contingency_table = np.array([present_counts, absent_counts])
    score = chi2_statistic(feature_contingency_table)
    chi2_scores[word] = score

print("Chi-squared scores for each word:")
for word, score in chi2_scores.items():
    print(f"{word}: {score:.2f}")

Chi-squared scores for each word:
composition: 50.45
gravity: 61.80
differential: 60.93
theory: 0.44


# Exercise 2 HMM Theory 2 Points
1. What are the assumptions made for a Hidden Markov Model? (0.5 Points)
2. Draw the graph of a Hidden Markov model. Explain the graph. (1 Point)
3. What are the limitations of HMMs? (0.5 Points)

### Answer
1. HMM assumptions:
- Markov Assumption: The current state depends only on the previous state. It is assumed that the future state is independent of all past states, given the present state. 
- Output Independence Assumption: The current observation is only dependent on the current state. The observation at a specific time is independent of all other states and observations, given the current state. 
2. <img src="HMM.png" alt="graph" style="width: 200px;"/>
- Hidden States (x): These are the unobservable states of the system. We cannot directly see which state the model is in. The initial state probabilities define the probability of starting in each hidden state.
- Observable States (y): These are the outputs or observations that we can see.
- Transitions (a): The arrows between hidden states represent the transition probabilities, which define the likelihood of moving from one hidden state to another.
- Emissions (b): The arrows from hidden states to observable states represent the emission probabilities, which define the likelihood of observing a particular output given a hidden state.
3. HMMs' limitations:
- Markov Assumption is too strong: The assumption that the current state only depends on the previous state is often an oversimplification for many real-world scenarios where longer-term dependencies exist.
- Independence of Observations: The assumption that observations are independent of each other, given the current state, can be restrictive. In many applications, observations may have dependencies among themselves.
- Fixed Number of States: The number of hidden states must be determined before training the model, which can be a difficult and critical decision.
- Lack of Contextual Information: HMMs do not inherently model any higher-level contextual or structural information beyond the immediate state-to-state transitions.

# Exercise 3 Named Entity Recognition 3 Points
In this exercise you will implement the Hidden Markov model for Named Entity Recognition.
1. Define the states and observations from the training data and compute the initial probabilities, i.e the probability of starting in a state. (1 Point)

In [4]:
lines = open("data/eng.train").readlines()

sentences = []
temp = []

for line in lines:

    if line == "\n":
        sentences.append(temp)
        temp = []
        continue

    toks = line.split()

    temp.append((toks[0], toks[3]))

print(sentences[:5])

[[('-DOCSTART-', 'O')], [('EU', 'I-ORG'), ('rejects', 'O'), ('German', 'I-MISC'), ('call', 'O'), ('to', 'O'), ('boycott', 'O'), ('British', 'I-MISC'), ('lamb', 'O'), ('.', 'O')], [('Peter', 'I-PER'), ('Blackburn', 'I-PER')], [('BRUSSELS', 'I-LOC'), ('1996-08-22', 'O')], [('The', 'O'), ('European', 'I-ORG'), ('Commission', 'I-ORG'), ('said', 'O'), ('on', 'O'), ('Thursday', 'O'), ('it', 'O'), ('disagreed', 'O'), ('with', 'O'), ('German', 'I-MISC'), ('advice', 'O'), ('to', 'O'), ('consumers', 'O'), ('to', 'O'), ('shun', 'O'), ('British', 'I-MISC'), ('lamb', 'O'), ('until', 'O'), ('scientists', 'O'), ('determine', 'O'), ('whether', 'O'), ('mad', 'O'), ('cow', 'O'), ('disease', 'O'), ('can', 'O'), ('be', 'O'), ('transmitted', 'O'), ('to', 'O'), ('sheep', 'O'), ('.', 'O')]]


In [10]:
# ===========================
# your code here
# ===========================
from collections import defaultdict
all_sets = set()
all_observations = set()
for sentence in sentences:
    for word, label in sentence:
        all_sets.add(word)
        all_observations.add(label)

print(f"Total unique words: {len(all_sets)}")
print(f"Total unique labels: {len(all_observations)}")

# Calculate the initial prior probabilities
initial_counts = defaultdict(int)
for sentence in sentences:
    if sentence:
        first_tag = sentence[0][1]
        initial_counts[first_tag] += 1

total_initial = sum(initial_counts.values())
initial_priors = {tag: count / total_initial for tag, count in initial_counts.items()}
print("Initial prior probabilities:")
for tag, prior in initial_priors.items():
    print(f"{tag}: {prior:.4f}")

Total unique words: 23624
Total unique labels: 8
Initial prior probabilities:
O: 0.6072
I-ORG: 0.1638
I-PER: 0.0900
I-LOC: 0.1055
I-MISC: 0.0335


2. Compute the probability of transitioning from one state to another $ P(\pi_i|\pi_{i-1})$. Define `<start>` as the start state and `<end>` as the end state i.e. start and end of each sentence when computing the transition counts. (1 Point)

In [ ]:
transition_counts = defaultdict(lambda: defaultdict(int))
transition_probs = defaultdict(dict)
tag_counts = defaultdict(int)
# Count transitions

# ===========================
# your code here
# ===========================

for sentence in sentences:
    if not sentence:
        continue

    first_tag = sentence[0][1]
    transition_counts[first_tag]['<s>'] += 1  # Count start transitions
    tag_counts[first_tag] += 1

    for i in range(len(sentence) - 1):
        current_tag = sentence[i][1]
        next_tag = sentence[i + 1][1]
        transition_counts[current_tag][next_tag] += 1
        tag_counts[current_tag] += 1

    last_tag = sentence[-1][1]
    transition_counts[last_tag]['</s>'] += 1  # Count end transitions
    tag_counts[last_tag] += 1

# Convert counts to probabilities

# ===========================
# your code here
# ===========================
for current_tag, next_tags in transition_counts.items():
    total_transitions = tag_counts[current_tag]
    for next_tag, count in next_tags.items():
        transition_probs[current_tag][next_tag] = count / total_transitions if total_transitions > 0 else 0.0

print("Transition probs: ", transition_probs)
for current_tag, next_tags in transition_probs.items():
    print(f"{current_tag} -> {next_tags}")


Transition probs:  defaultdict(<class 'dict'>, {'O': {'<s>': 0.05066138155257649, '</s>': 0.07943815971139714, 'I-MISC': 0.015916581303166616, 'O': 0.7732040261880372, 'I-ORG': 0.021110764708502205, 'I-PER': 0.028854718745824612, 'I-LOC': 0.03080323342092371, 'B-MISC': 1.1134369571994834e-05}, 'I-ORG': {'<s>': 0.19709377007064868, 'O': 0.4901252408477842, 'I-ORG': 0.29736673089274246, '</s>': 0.013567758509955041, 'I-MISC': 0.000722543352601156, 'I-PER': 0.0004816955684007707, 'I-LOC': 0.00016056518946692356, 'B-ORG': 0.0004816955684007707}, 'I-MISC': {'O': 0.6204033214709371, 'I-MISC': 0.22380387504942664, '<s>': 0.0992487149070779, 'I-ORG': 0.00771055753262159, 'I-LOC': 0.0019770660340055358, 'B-MISC': 0.006919731119019375, '</s>': 0.027876631079478055, 'I-PER': 0.012060102807433769}, 'I-PER': {'<s>': 0.10811893884747936, 'I-PER': 0.3629077502604793, '</s>': 0.019395688066041517, 'O': 0.5092570329406108, 'I-MISC': 8.014747134727899e-05, 'I-LOC': 0.00024044241404183698}, 'I-LOC': {'<s

3. Compute the emission probabilities, i.e. the probability of seeing a observation in a given state. (1 Point)

In [14]:
# Initialize emission probability dict
emission_counts = defaultdict(lambda: defaultdict(int))
emission_probs = defaultdict(dict)

# Count emissions

# ===========================
# your code here
# ===========================
for sentence in sentences:
    for word, tag in sentence:
        emission_counts[tag][word] += 1

# Convert counts to probabilities
for tag, words in emission_counts.items():
    total_count_for_tag = tag_counts[tag]
    for word, count in words.items():
        emission_probs[tag][word] = count / total_count_for_tag if total_count_for_tag > 0 else 0.0
# ===========================
# your code here
# ===========================
print("Emission probabilities:")
for tag, words in emission_probs.items():
    print(f"{tag}: {words}")

Emission probabilities:
O: {'-DOCSTART-': 0.0052665568075535565, 'rejects': 5.567184785997417e-06, 'call': 0.00016701554357992251, 'to': 0.018828218946243265, 'boycott': 2.7835923929987084e-05, 'lamb': 1.670155435799225e-05, '.': 0.04098561439451298, '1996-08-22': 0.0006958980982496771, 'The': 0.006084932971095177, 'said': 0.010277023114951232, 'on': 0.01133478822429074, 'Thursday': 0.0015866476640092639, 'it': 0.0030786531866565715, 'disagreed': 1.1134369571994834e-05, 'with': 0.00467643522023783, 'advice': 2.7835923929987084e-05, 'consumers': 5.0104663073976754e-05, 'shun': 5.567184785997417e-06, 'until': 0.0002950607936578631, 'scientists': 3.34031087159845e-05, 'determine': 6.123903264597158e-05, 'whether': 0.00025052331536988375, 'mad': 7.237340221796642e-05, 'cow': 6.6806217431969e-05, 'disease': 0.0001948514675099096, 'can': 0.0003952701198058166, 'be': 0.002922772012648644, 'transmitted': 1.1134369571994834e-05, 'sheep': 6.6806217431969e-05, "'s": 0.008328508439852136, 'represe

# Exercise 4 Bayesian Networks 1 Points
1. Draw the graphs representing the following distributions (0.5 points)
   
i) P(A,B,C,D) = P(A)P(C)P(B|C)P(D|C,A)\
ii) P(A,B,C,D) = P(D)P(B)P(A|B,C)P(C|D)\
iii) P(A,B,C) = P(A|B)P(B|C)P(C)

i)

<img src="ex4.1_i).png" alt="graph" style="width: 200px;"/>

ii)

<img src="ex4.1_ii).png" alt="graph" style="width: 200px;"/>

iii)

<img src="ex4.1_iii).png" alt="graph" style="width: 200px;"/>

2. Give the joint distribution of the following graphs (0.5 points)

i)

<img src="ex4.2_i).png" alt="graph" style="width: 200px;"/>

ii)

<img src="ex4.2_ii).png" alt="graph" style="width: 200px;"/>

iii)

<img src="ex4.2_iii).png" alt="graph" style="width: 200px;"/>

i) P(A,B,C,D) = P(A)P(B|A)P(C|A)P(D|B,C)

ii) P(A,B,C,D) = P(C)P(A|C)P(B|A)P(D|B)

iii) P(A,B,C,D) = P(A)P(C)P(D|C)P(B|A,D)

# Exercise 5 Conditional random fields 3 Points
1. While HMMs are generative models CRFs are discriminativ. Explain why? (0.5 points)
2. What are the advantages of CRFs over HMMs? (0.5 points)

### Answer:
1. HMMs are generative models because they model the joint probability of observations and labels. This means HMMs define how the observations and labels are generated together, and they require modeling the distribution of the inputs. CRFs are discriminative models because they model the conditional probability of the labels given the observations. They focus directly on learning the decision boundary between classes without needing to model how the data is generated. This allows CRFs to incorporate overlapping, non-independent features of the input without making assumptions about their distribution.
2. CRFs have several advantages over HMMs:
- No strong independence assumptions: CRFs do not assume that observations are conditionally independent given the labels, unlike HMMs.
- More flexible feature representation: CRFs allow the use of rich and overlapping features (e.g., context windows) for better modeling.
- Better performance in structured prediction: CRFs often outperform HMMs in tasks like part-of-speech tagging or named entity recognition, especially with complex input data.
- Discriminative training: By directly modeling $P(Y∣X)$, CRFs focus on prediction accuracy rather than data generation, leading to better generalization.


   
Now you will use the `sklearn_crfsuite` to implement a CRF for Part of Speech tagging. You'll be using the Treebank corpus for that. Run the following cell to get the data.

In [1]:
pip install sklearn-crfsuite # install this please

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [ ]:
import nltk
nltk.download('treebank')
corpus = nltk.corpus.treebank.tagged_sents()[:-100]
test_data =  nltk.corpus.treebank.tagged_sents()[-100:]

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\WangEntang\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


3. Complete the function `word2features`, `sent2features` and `sent2labels` .Set up the `word2features` such that the following features are used in your code: (if you're not sure how to do it, have a look at the example in the next block): (1 point)
- The word itself.
- A boolean indicating if the word is the first word in the sentence.
- A boolean indicating if the word is the last word in the sentence.
- A boolean indicating if the first character of the word is capitalized.
- A boolean indicating if the word is all in uppercase letters.
- A boolean indicating if the word is all in lowercase letters.
- The first character of the word.
- The first two characters of the word.
- The first three characters of the word.
- The last character of the word.
- The last two characters of the word.
- The last three characters of the word.
- The previous word in the sentence. If the word is the first word in the sentence, this feature is an empty string.
- The next word in the sentence. If the word is the last word in the sentence, this feature is an empty string.
- A boolean indicating if the word contains a hyphen.
- A boolean indicating if the word is a number.
- A boolean indicating if the word has any capitalized letters other than the first letter.


In [ ]:
# def word2features_example(sent, i):
#     word = sent[i]

#     features = {
#         'word_length': len(word),  # The length of the word
#         'is_alphabet': word.isalpha(),  # Is the word composed of alphabets only
#         'is_digit': word.isdigit(),  # Is the word composed of digits only
#         'is_alphanumeric': word.isalnum(),  # Is the word composed of alphabets and numbers only
#         'is_uppercase': word.isupper(),  # Is the word in uppercase
#         'is_lowercase': word.islower(),  # Is the word in lowercase
#         'prev_word': '' if i == 0 else sent[i - 1],  # The previous word
#         'next_word': '' if i == len(sent) - 1 else sent[i + 1],  # The next word
#     }

#     return features

In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

train_sents= corpus

def word2features(sent, i):
    """
    Generate features for a given word in a sentence.

    Args:
    sent (list): A list of tuples representing a sentence, where each tuple contains a word and its label.
    i (int): The index of the word in the sentence for which features are to be generated.

    Returns:
    dict: A dictionary where keys are feature names and values are corresponding feature values.
    """
    word = sent[i][0]

    features = {
    'word': word, # The word itself
    'is_first': i == 0, # Is the first word in the sentence
    'is_last': i == len(sent) - 1, # Is the last word in the sentence
    'is_capitalized': word[0].isupper(), # Is the first character capitalized
    'is_uppercase': word.isupper(), # Is the word all in uppercase letters
    'is_lowercase': word.islower(), # Is the word all in lowercase letters
    'first_char': word[0], # The first character of the word
    'first_two_chars': word[:2], # The first two characters of the word
    'first_three_chars': word[:3], # The first three characters of the word
    'last_char': word[-1], # The last character of the word
    'last_two_chars': word[-2:], # The last two characters of the word
    'last_three_chars': word[-3:], # The last three characters of the word
    'prev_word': '' if i == 0 else sent[i - 1][0], # The previous word in the sentence
    'next_word': '' if i == len(sent) - 1 else sent[i + 1][0], # The next word in the sentence
    'contains_hyphen': '-' in word, # Does the word contain a hyphen
    'is_number': word.isdigit(), # Is the word a number
    'has_capitalized_other_than_first': any(c.isupper() for c in word[1:]), # Does the word have any capitalized letters other than the first letter
    }

    return features


def sent2features(sent):
    """
    Generate features for all words in a sentence.

    Args:
    sent (list): A list of tuples representing a sentence, where each tuple contains a word and its label.

    Returns:
    list: A list of feature dictionaries. Each dictionary corresponds to the features of a word in the sentence.
    """
    features_list = []
    for i in range(len(sent)):
        features_list.append(word2features(sent, i))

    return features_list


def sent2labels(sent):
    """
    Extract labels from a sentence.

    Args:
    sent (list): A list of tuples representing a sentence, where each tuple contains a word and its label.

    Returns:
    list: A list of labels extracted from the sentence.
    """
    return [label for _, label in sent]

4. Train the CRF and run the evaluation code. For that look up the documentation of [sklearn_crfsuite](https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html#training) on how to set up the training code. (1 point).

In [ ]:
#Train the CRF
from sklearn_crfsuite import CRF

# Initialize the CRF model
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=True)
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]
print(X_train[:10])
print(y_train[:10])
crf.fit(X_train, y_train)

[[{'word': 'Pierre', 'is_first': True, 'is_last': False, 'is_capitalized': True, 'is_uppercase': False, 'is_lowercase': False, 'first_char': 'P', 'first_two_chars': 'Pi', 'first_three_chars': 'Pie', 'last_char': 'e', 'last_two_chars': 're', 'last_three_chars': 'rre', 'prev_word': '', 'next_word': 'Vinken', 'contains_hyphen': False, 'is_number': False, 'has_capitalized_other_than_first': False}, {'word': 'Vinken', 'is_first': False, 'is_last': False, 'is_capitalized': True, 'is_uppercase': False, 'is_lowercase': False, 'first_char': 'V', 'first_two_chars': 'Vi', 'first_three_chars': 'Vin', 'last_char': 'n', 'last_two_chars': 'en', 'last_three_chars': 'ken', 'prev_word': 'Pierre', 'next_word': ',', 'contains_hyphen': False, 'is_number': False, 'has_capitalized_other_than_first': False}, {'word': ',', 'is_first': False, 'is_last': False, 'is_capitalized': False, 'is_uppercase': False, 'is_lowercase': False, 'first_char': ',', 'first_two_chars': ',', 'first_three_chars': ',', 'last_char': 

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    max_iterations=100)

In [ ]:
#Evaluation code
X_test = [sent2features(s) for s in test_data]
y_test = [sent2labels(s) for s in test_data]
y_pred = crf.predict(X_test)
labels=list(crf.classes_)

metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

C:\Users\WangEntang\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.970868015776912